In [15]:
import json
import time

import numpy as np
import pandas as pd
import tensorflow as tf
import _pickle as cPickle

from enum import Enum
from collections import Counter
from models import EfficientCapsNet
from utils.tools import get_save_path
from utils import DataDEAP, generate_tf_data, pre_process
from preprocess import preprocess

In [16]:
NUM_CHANNELS = 19

# gpus = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
# tf.config.experimental.set_memory_growth(gpus[0], True)

In [17]:
# tf.config.experimental.list_physical_devices('GPU')

In [18]:
def apply_mixup(data):
    data_in = data.transpose(0,2,1)
    
    window_size = 128

    # 0 valence, 1 arousal, 2 dominance, 3 liking   
    # dimensions = ['valence', 'arousal', 'dominance', 'liking']

    data_inter_rnn	= np.empty([0, window_size, NUM_CHANNELS])

    trials = data_in.shape[0]

    # Data pre-processing
    for trial in range(0,trials):
        base_signal = (data_in[trial,0:128,:] + data_in[trial,128:256,:] + data_in[trial,256:384,:])/3

        data = data_in[trial,384:,:]

        # compute the deviation between baseline signals and experimental signals
        for i in range(0,60):
            data[i*128:(i+1)*128,:]= data[i*128:(i+1)*128,:] - base_signal

        #read data and label
        data = norm_dataset(data)
        data = segment_signal_without_transition(data, window_size)

        # rnn data process
        data_rnn    = data.reshape(int(data.shape[0]/window_size), window_size, NUM_CHANNELS)
        # append new data and label
        data_inter_rnn  = np.vstack([data_inter_rnn, data_rnn])

    return data_inter_rnn

def windows(data, size):
	start = 0
	while ((start+size) < data.shape[0]):
		yield int(start), int(start + size)
		start += size

def segment_signal_without_transition(data, window_size):
	# get data file name and label file name
	for (start, end) in windows(data, window_size):
		if((len(data[start:end]) == window_size)):
			if(start == 0):
				segments = data[start:end]
				segments = np.vstack([segments, data[start:end]])
			else:
				segments = np.vstack([segments, data[start:end]])
	return segments

def feature_normalize(data):
	mean = data[data.nonzero()].mean()
	sigma = data[data. nonzero ()].std()
	data_normalized = data
	data_normalized[data_normalized.nonzero()] = (data_normalized[data_normalized.nonzero()] - mean)/sigma
	# return shape: 9*9
	return data_normalized

def norm_dataset(dataset_1D):
	norm_dataset_1D = np.zeros([dataset_1D.shape[0], NUM_CHANNELS])
	for i in range(dataset_1D.shape[0]):
		norm_dataset_1D[i] = feature_normalize(dataset_1D[i])
	# return shape: m*32
	return norm_dataset_1D

In [19]:
def pre_process(data):
	return (data)[...,None].astype('float32')

def generator(image):
	return image

PARALLEL_INPUT_CALLS = 16
def generate_tf_data(data, batch_size):
    dataset_test = pre_process(data)
    dataset_test = tf.data.Dataset.from_tensor_slices(dataset_test)
    dataset_test = dataset_test.cache()
    # dataset_test = dataset_test.shuffle(buffer_size=DEAP_TRAIN_IMAGE_COUNT)
    dataset_test = dataset_test.map(generator,
        num_parallel_calls=PARALLEL_INPUT_CALLS)
    dataset_test = dataset_test.batch(batch_size)
    dataset_test = dataset_test.prefetch(-1)

    return dataset_test

In [20]:
import numba; 

try:
    numba.njit(lambda x: x + 1)(123)
except:
    print()

In [21]:
preprocess("inputs\A210223012007.EDF", "inputs\preprocessed\A210223012007.dat")

Extracting EDF parameters from c:\Users\aadim\workspace\NRPU\NRPU-Demo\inputs\A210223012007.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 37887  =      0.000 ...   147.996 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 213 samples (1.664 sec)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.


In [22]:
def load_config(config_path):
    """
    Load config file
    """
    with open(config_path) as json_data_file:
        config = json.load(json_data_file)
    return config

config = load_config('config.json')

In [23]:
def predict(data_path, model_path):
    subject = cPickle.load(open(data_path, 'rb'), encoding='latin1')
    data = subject['data']
    data = data[np.newaxis, :]
    print(f"Data Shape: {data.shape}")
    data_rnn = apply_mixup(data)[1:]
    print(f"Data RNN Shape: {data_rnn.shape}")

    test_data = generate_tf_data(data_rnn, 16)
    # print(f"Test data Shape: {tf.shape(test_data.as_numpy_iterator())}")
    for element in test_data:
        print(element.shape)
        break
    model_test = EfficientCapsNet("DEAP", config, mode='test', custom_path=model_path, verbose=False)
    model_test.load_graph_weights()
    predictions = list()
    pred = model_test.predict(test_data)
    pred = np.argmax(pred, 1)
    return pred

In [24]:
data = predict("inputs\preprocessed\A210223012007.dat", "bin\\sub_ind\\best_fold.h5")
data

Data Shape: (1, 19, 8066)
Data RNN Shape: (60, 128, 19)
(16, 128, 19, 1)


array([2, 0, 2, 2, 0, 2, 2, 7, 1, 7, 0, 2, 0, 2, 2, 7, 2, 2, 2, 0, 1, 0,
       2, 0, 7, 0, 2, 1, 0, 0, 0, 4, 2, 2, 0, 2, 6, 1, 2, 2, 0, 4, 4, 2,
       0, 1, 7, 2, 0, 0, 2, 7, 2, 2, 2, 2, 0, 2, 2, 0], dtype=int64)

In [25]:
np.bincount(data).argmax()  # max freq

2

In [28]:
def class_to_vad(labels):
    label_map = {
        0: 'HVHAHD',
        1: 'HVHALD',
        2: 'HVLAHD',
        3: 'HVLALD',
        4: 'LVHAHD',
        5: 'LVHALD',
        6: 'LVLAHD',
        7: 'LVLALD'
    }

    return [label_map[num] for num in labels]

vad_labels = class_to_vad(data)



In [31]:
def vad_to_emotion(vad_labels):
    emotion_map = {
        'HVHAHD': 'Joy',
        'HVHALD': 'Surprise',
        'HVLAHD': 'Excited',
        'HVLALD': 'Calm',
        'LVHAHD': 'Anger',
        'LVHALD': 'Fear', #Disgust
        'LVLAHD': 'Disgust',
        'LVLALD': 'Sadness'
    }

    return [emotion_map[num] for num in vad_labels]

emotion_labels = vad_to_emotion(vad_labels)

In [38]:
def dominant_emotion(emotion_labels):
    return Counter(emotion_labels).most_common(1)[0][0]

dominant_emotion(emotion_labels)

'Excited'

In [39]:
import plotly.express as px

fig = px.histogram(emotion_labels)
fig.show()

In [45]:
df = pd.DataFrame(emotion_labels, columns=['Emotions'])

In [54]:
fig = px.scatter(df, labels={
                     "index": "Time (sec)",
                     "value": "Emotions"
                 }, title="Emotion Classification")
fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(showlegend=False)
fig.show()

In [79]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.6, 0.4],
    subplot_titles=("Emotion Labels", "Emotion Frequency"),
    specs=[[{"type": "scatter"}, {"type": "bar"}]])

fig.add_trace(
    go.Scatter(x=df.index, y=df.Emotions,
                    mode='lines+markers',
                    showlegend=False), row=1, col=1)

# fig.add_trace(
#     go.Scatter(df, showlegend=False,
#                   marker=dict(color="crimson", size=4, opacity=0.8)),
#     row=1, col=1
# )

# Add locations bar chart
fig.add_trace(
    go.Histogram(x=df.Emotions, showlegend=False),
    row=1, col=2
)


# Set theme, margin, and annotation in layout
fig.update_layout(
    # template="plotly_dark",
    margin=dict(r=10, t=40, b=10, l=10)
    # title_text='Emotion Labels' # title of plot
)


In [59]:
df.index

RangeIndex(start=0, stop=60, step=1)

In [81]:
subject = cPickle.load(open("inputs\preprocessed\A210223012007.dat", 'rb'), encoding='latin1')
data = subject['data']
data = data[np.newaxis, :]
print(f"Data Shape: {data.shape}")

Data Shape: (1, 19, 8066)


In [85]:
(data.shape[-1] // 128) - 3

60